In [1]:
import pandas as pd
import sys
sys.path.append("../scripts")
from db_utils import table_exists, read_query_as_df

In [2]:
stock_df = read_query_as_df("SELECT * FROM stock_prices") if table_exists("stock_prices") else pd.DataFrame()
sentiment_df = read_query_as_df("SELECT * FROM sentiment_scores") if table_exists("sentiment_scores") else pd.DataFrame()


In [3]:
if ('date' in stock_df.columns and 'date' in sentiment_df.columns):
    # Convert the 'date' columns to datetime.date objects
    stock_df['date'] = pd.to_datetime(stock_df['date']).dt.date
    sentiment_df['date'] = pd.to_datetime(sentiment_df['date']).dt.date

In [4]:
sentiment_agg = sentiment_df.groupby(['stock', 'date']).agg({
    'polarity': 'mean',
    'subjectivity': 'mean'
}).reset_index()
sentiment_agg.head()

,stock,date,polarity,subjectivity
0,AAPL,2017-01-31,0.000000,0.0
1,AAPL,2017-05-03,0.000000,0.0
2,AAPL,2017-05-23,0.000000,0.3
3,AAPL,2017-07-29,0.500000,1.0
4,AAPL,2018-02-01,0.216667,0.4


In [5]:
merged_df = pd.merge(stock_df, sentiment_agg, on=['stock', 'date'], how='inner')

In [6]:
merged_df.shape

(48, 9)

In [7]:
merged_df['daily_return'] = merged_df.groupby('stock')['close'].pct_change()

In [8]:
merged_df['sentiment_spike'] = merged_df['polarity'] > 0.5
merged_df['neg_sentiment'] = merged_df['polarity'] < -0.3

In [9]:
merged_df['vol_spike'] = merged_df.groupby('stock')['volume'].transform(
    lambda x: x > x.rolling(7).mean() * 1.5
)

In [10]:
merged_df['day_of_week'] = pd.to_datetime(merged_df['date']).dt.day_name()

In [11]:
merged_df['sentiment_label'] = merged_df['polarity'].apply(lambda x: "Positive" if x > 0.05 else "Negative" if x < -0.05 else "Neutral")


In [12]:
def detect_spike(group):
    # Exclude 'stock' column from being recalculated if it's included
    group = group.copy()
    
    group['rolling_mean'] = group['polarity'].rolling(window=3, min_periods=2).mean()
    group['rolling_std'] = group['polarity'].rolling(window=3, min_periods=2).std()
    group['sentiment_spike'] = (abs(group['polarity'] - group['rolling_mean']) > 0.6 * group['rolling_std'])
    return group

# Apply the function to each ticker
merged_df = merged_df.sort_values(['stock', 'date'])  # just in case
grouped = [detect_spike(group) for _, group in merged_df.groupby('stock')]
merged_df = pd.concat(grouped).reset_index(drop=True)

In [13]:
merged_df['spike_direction'] = merged_df.apply(
    lambda row: 'Positive Spike' if row['sentiment_spike'] and row['polarity'] > 0 
    else 'Negative Spike' if row['sentiment_spike'] and row['polarity'] < 0 
    else 'None', axis=1
)

print(merged_df.groupby('spike_direction')['daily_return'].describe())

                 count      mean       std       min       25%       50%  \
spike_direction                                                            
Negative Spike     8.0 -0.023055  0.046901 -0.131170 -0.026810 -0.012431   
None              20.0 -0.015030  0.043821 -0.085630 -0.050767 -0.014787   
Positive Spike    15.0 -0.006973  0.045758 -0.095157 -0.030413 -0.006651   

                      75%       max  
spike_direction                      
Negative Spike  -0.000887  0.020041  
None             0.005887  0.070033  
Positive Spike   0.012462  0.081984  


In [14]:
merged_df['rolling_sentiment'] = merged_df.groupby('stock')['polarity'].transform(lambda x: x.rolling(7).mean())

In [15]:
def label_spike(row):
    if row['sentiment_spike'] and row['vol_spike']:
        return "Both"
    elif row['sentiment_spike']:
        return "Sentiment"
    elif row['vol_spike']:
        return "Volume"
    else:
        return "None"

merged_df['spike_type'] = merged_df.apply(label_spike, axis=1)

In [16]:
merged_df.head()

,date,close,high,low,open,volume,stock,polarity,subjectivity,daily_return,sentiment_spike,neg_sentiment,vol_spike,day_of_week,sentiment_label,rolling_mean,rolling_std,spike_direction,rolling_sentiment,spike_type
0,2025-02-24,247.100006,248.860001,244.419998,244.929993,51326400,AAPL,0.000000,0.000000,NaN,False,False,False,Monday,Neutral,NaN,NaN,None,NaN,None
1,2025-02-27,237.300003,242.460007,237.059998,239.410004,41153600,AAPL,0.157143,0.484821,-0.039660,True,False,False,Thursday,Positive,0.078571,0.111117,Positive Spike,NaN,Sentiment
2,2025-03-12,216.979996,221.750000,214.910004,220.139999,62547500,AAPL,0.100000,0.400000,-0.085630,False,False,False,Wednesday,Positive,0.085714,0.079539,None,NaN,None
3,2025-03-19,215.240005,218.759995,213.750000,214.220001,54385400,AAPL,-0.071429,0.214286,-0.008019,True,False,False,Wednesday,Negative,0.061905,0.118952,Negative Spike,NaN,Sentiment
4,2025-03-20,214.100006,217.490005,212.220001,213.990005,48862900,AAPL,0.266886,0.458613,-0.005296,True,False,False,Thursday,Positive,0.098486,0.169162,Positive Spike,NaN,Sentiment


In [17]:
merged_df.to_csv("../data/candlethrob_dataset.csv", index=False, )

In [20]:
query = """
SELECT
  s.stock,
  DATE(s.date) AS date,
  COUNT(s.stock) AS total_mentions,
  ROUND(AVG(s.polarity), 3) AS avg_sentiment,
  ROUND(AVG(s.subjectivity), 3) AS avg_subjectivity,
  sp.open,
  sp.close,
  (sp.close - sp.open) AS price_change,
  sp.volume,
  COUNT(s.stock) * ROUND(AVG(s.polarity), 3) AS signal_strength
FROM sentiment_scores s
LEFT JOIN stock_prices sp
  ON s.stock = sp.stock AND DATE(s.date) = DATE(sp.date)
GROUP BY s.stock, DATE(s.date)
ORDER BY date;
"""

df = read_query_as_df(query)
df.to_csv("../data/candlethrob_analysis.csv", index=False)
